# Проект: классификация

### Блок 1: Код выполнения

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler
from sklearn.feature_selection import SelectKBest, f_classif

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import (
    RandomForestClassifier,
    GradientBoostingClassifier,
    StackingClassifier,
)

from sklearn.metrics import (
    accuracy_score,
    f1_score,
    recall_score,
    precision_score,
)

import optuna


## Часть 1. Знакомство с данными, обработка пропусков и выбросов

### Блок 2: Код выполнения

In [ ]:
df = pd.read_csv('bank_fin\\bank_fin.csv', sep = ';')

### Блок 3: Код выполнения

In [ ]:
missing_per_column = df.isnull().sum()

missing_per_column = missing_per_column[missing_per_column > 0]

print(f"Столбцы с пропущенными значениями и их количество: {missing_per_column}")

### Блок 4: Код выполнения

In [ ]:
for job in df['job'].unique():
    print(job)

### Блок 5: Код выполнения

In [ ]:
df['balance'] = df['balance'].astype(str) \
    .str.replace(' ', '') \
    .str.replace(',', '.') \
    .str.replace('$', '') \
    .astype(float)

mean_balance = round(df['balance'].mean(), 3)

formatted_balance = f"{mean_balance:.3f}".rstrip('0').rstrip('.')

print(f'Среднее значение с округлением до 3 знаков: {formatted_balance}')


### Блок 6: Код выполнения

In [ ]:
df['balance'] = pd.to_numeric(df['balance'], errors='coerce')

median_balance = df['balance'].median()
print(f"Медианное значение баланса: {median_balance}")

df['balance'] = df['balance'].fillna(median_balance)

mean_balance = round(df['balance'].mean(), 3)
print(f"Среднее значение баланса: {mean_balance}")


### Блок 7: Код выполнения

In [ ]:
job_mode = df.loc[df['job'] != 'unknown', 'job'].mode()[0]
df['job'] = df['job'].replace('unknown', job_mode)

education_mode = df.loc[df['education'] != 'unknown', 'education'].mode()[0]
df['education'] = df['education'].replace('unknown', education_mode)

popular_clients = df[(df['job'] == job_mode) & (df['education'] == education_mode)]

mean_balance_popular = round(popular_clients['balance'].mean(), 3)
print(f'Средний баланс популярных клиентов: {mean_balance_popular}')

### Блок 8: Код выполнения

In [ ]:
Q1 = df['balance'].quantile(0.25)
Q3 = df['balance'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

print(f'Нижняя граница: {lower_bound}')
print(f'Верхняя граница: {upper_bound}')

df = df[(df['balance'] >= lower_bound) & (df['balance'] <= upper_bound)]

## Часть 2:  Разведывательный анализ

### Блок 9: Код выполнения

In [ ]:
deposit_counts = df['deposit'].value_counts()
print(deposit_counts)

plt.figure(figsize=(6,4))
bars = plt.bar(deposit_counts.index, deposit_counts.values)
plt.title('Распределение целевой переменной (депозит открыт/не открыт)', fontsize=14)
plt.xlabel('Депозит оформлен', fontsize=12)
plt.ylabel('Число клиентов', fontsize=12)

for bar in bars:
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2, yval + 50, int(yval), ha='center', va='bottom', fontsize=11, fontweight='bold')

plt.xticks(fontsize=11)
plt.yticks(fontsize=11)
plt.tight_layout()
plt.show()

### Задания 2 и 3

### Блок 10: Код выполнения

In [ ]:
quant_cols = df.select_dtypes(include='number').columns
desc_stats = df[quant_cols].describe().T
desc_stats['range'] = desc_stats['max'] - desc_stats['min']
desc_stats[['count', 'mean', 'std', 'min', '25%', '50%', '75%', 'max', 'range']]
print('Максимальный возраст клиента банка:', df['age'].max())
print('Минимальная продолжительность разговора с клиентом (сек):', df['duration'].min())



for col in quant_cols:
    plt.figure(figsize=(12,4))
    
    plt.subplot(1,2,1)
    plt.hist(df[col], bins=30)
    plt.title(f'Гистограмма: {col}')
    plt.xlabel(col)
    plt.ylabel('Частота')
    
    plt.subplot(1,2,2)
    plt.boxplot(df[col], vert=False)
    plt.title(f'Boxplot: {col}')
    plt.xlabel(col)
    
    plt.tight_layout()
    plt.show()


### Задания 4 и 5

### Блок 11: Код выполнения

In [ ]:
job_unique = df['job'].nunique()
print('Всего сфер занятости:', job_unique)
print('Сферы занятости:', df['job'].unique())

plt.figure(figsize=(10,5))
df['job'].value_counts().plot(kind='bar')
plt.title('Распределение клиентов по профессиям')
plt.xlabel('Профессия')
plt.ylabel('Количество клиентов')
plt.xticks(rotation=30)
plt.tight_layout()
plt.show()

months = df['month'].unique()
print('Месяцы, в которые была кампания:', sorted(months))
print('Всего месяцев:', len(months))

plt.figure(figsize=(8,4))
df['month'].value_counts().loc[sorted(df['month'].unique())].plot(kind='bar')
plt.title('Число контактов по месяцам')
plt.xlabel('Месяц')
plt.ylabel('Количество контактов')
plt.xticks(rotation=30)
plt.tight_layout()
plt.show()

marital_mode = df['marital'].mode()[0]
print('Самое частое семейное положение:', marital_mode)
print(df['marital'].value_counts())

plt.figure(figsize=(6,4))
df['marital'].value_counts().plot(kind='bar')
plt.title('Распределение по семейному положению')
plt.xlabel('Семейное положение')
plt.ylabel('Количество клиентов')
plt.tight_layout()
plt.show()

education_mode = df['education'].mode()[0]
print('Самый частый уровень образования:', education_mode)
print(df['education'].value_counts())

plt.figure(figsize=(7,4))
df['education'].value_counts().plot(kind='bar')
plt.title('Распределение по уровню образования')
plt.xlabel('Образование')
plt.ylabel('Количество клиентов')
plt.tight_layout()
plt.show()

### Блок 12: Код выполнения

In [ ]:
pivot = pd.crosstab(df['poutcome'], df['deposit'])
print(pivot)

success_poutcome = pivot[pivot['yes'] > pivot['no']]
print('Статусы, где успехов больше, чем неудач:')
print(success_poutcome)

pivot.plot(kind='bar', stacked=True, figsize=(8,5))
plt.title('Результаты текущей кампании по статусу предыдущей')
plt.xlabel('Статус предыдущей кампании')
plt.ylabel('Количество клиентов')
plt.xticks(rotation=30)
plt.tight_layout()
plt.show()

pivot = pd.crosstab(df['poutcome'], df['deposit'])
print('Таблица сопряжённости:\n', pivot, '\n')

success_poutcome = pivot[pivot['yes'] > pivot['no']]
print('Статусы предыдущей кампании, где успехов больше, чем неудач:')
print(success_poutcome)

pivot.plot(kind='bar', stacked=True, figsize=(8,5))
plt.title('Результаты текущей кампании по статусу предыдущей')
plt.xlabel('Статус предыдущей кампании')
plt.ylabel('Количество клиентов')
plt.xticks(rotation=30)
plt.tight_layout()
plt.show()



### Блок 13: Код выполнения

In [ ]:
month_stats = df.groupby('month')['deposit'].value_counts(normalize=True).unstack().fillna(0)

month_stats_no = month_stats['no']

worst_month = month_stats_no.idxmax()
max_percent = month_stats_no.max()

print(f'В месяц {worst_month} самый большой процент неудач: {max_percent:.2%}')


### Блок 14: Код выполнения

In [ ]:
def age_group(age):
    if age < 30:
        return '<30'
    elif 30 <= age < 40:
        return '30-40'
    elif 40 <= age < 50:
        return '40-50'
    elif 50 <= age < 60:
        return '50-60'
    else:
        return '60+'

df['age_group'] = df['age'].apply(age_group)

print(df['age_group'].value_counts())
age_deposit = df.groupby('age_group')['deposit'].value_counts(normalize=True).unstack().fillna(0)

age_deposit[['yes', 'no']].plot(kind='bar', stacked=True, figsize=(7,5), color=['#4caf50', '#f44336'])
plt.title('Доли открывших и не открывших депозит по возрастным группам')
plt.xlabel('Возрастная группа')
plt.ylabel('Доля клиентов')
plt.legend(['Открыли депозит', 'Не открыли депозит'])
plt.xticks(rotation=0)
plt.tight_layout()
plt.show()




### Задания 9 и 10

### Блок 15: Код выполнения

In [ ]:
marital_deposit = df.groupby('marital')['deposit'].value_counts(normalize=True).unstack().fillna(0)

marital_deposit[['yes', 'no']].plot(kind='bar', stacked=True, figsize=(7,5), color=['#4caf50', '#f44336'])
plt.title('Доли открывших и не открывших депозит по семейному статусу')
plt.xlabel('Семейное положение')
plt.ylabel('Доля клиентов')
plt.legend(['Открыли депозит', 'Не открыли депозит'])
plt.xticks(rotation=0)
plt.tight_layout()
plt.show()

### Блок 16: Код выполнения

In [ ]:

education_deposit = df.groupby('education')['deposit'].value_counts(normalize=True).unstack().fillna(0)

education_deposit[['yes', 'no']].plot(kind='bar', stacked=True, figsize=(7,5), color=['#4caf50', '#f44336'])
plt.title('Доли открывших и не открывших депозит по уровню образования')
plt.xlabel('Уровень образования')
plt.ylabel('Доля клиентов')
plt.legend(['Открыли депозит', 'Не открыли депозит'])
plt.xticks(rotation=0)
plt.tight_layout()
plt.show()


### Блок 17: Код выполнения

In [ ]:

job_deposit = df.groupby('job')['deposit'].value_counts(normalize=True).unstack().fillna(0)

job_deposit[['yes', 'no']].plot(kind='bar', stacked=True, figsize=(12,6), color=['#4caf50', '#f44336'])
plt.title('Доли открывших и не открывших депозит по профессиям')
plt.xlabel('Профессия')
plt.ylabel('Доля клиентов')
plt.legend(['Открыли депозит', 'Не открыли депозит'])
plt.xticks(rotation=30)
plt.tight_layout()
plt.show()

most_common_job = df['job'].mode()[0]
print('Сфера с наибольшим числом клиентов:', most_common_job)
count = df['job'].value_counts()[most_common_job]
print(f'Сфера с наибольшим числом клиентов: {most_common_job} ({count} клиентов)')


### Блок 18: Код выполнения

In [ ]:
df_yes = df[df['deposit'] == 'yes']
df_no = df[df['deposit'] == 'no']

pivot_yes = pd.crosstab(df_yes['education'], df_yes['marital'])
pivot_no = pd.crosstab(df_no['education'], df_no['marital'])

plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
sns.heatmap(pivot_yes, annot=True, fmt='d', cmap='Greens')
plt.title('Открыли депозит')
plt.xlabel('Семейное положение')
plt.ylabel('Уровень образования')

plt.subplot(1, 2, 2)
sns.heatmap(pivot_no, annot=True, fmt='d', cmap='Reds')
plt.title('Не открыли депозит')
plt.xlabel('Семейное положение')
plt.ylabel('Уровень образования')

plt.tight_layout()
plt.show()

print("Сводная таблица — Открыли депозит:")
print(pivot_yes)
print("\nСводная таблица — Не открыли депозит:")
print(pivot_no)

## Часть 3: преобразование данных

### Блок 19: Код выполнения

In [ ]:
le_edu = LabelEncoder()
df['education'] = le_edu.fit_transform(df['education'])
print('Сумма закодированных значений education:', df['education'].sum())
le_age = LabelEncoder()
df['age_group'] = le_age.fit_transform(df['age_group'])
print('Сумма закодированных значений age_group:', df['age_group'].sum())

### Задания 2 и 3

### Блок 20: Код выполнения

In [ ]:
df['deposit'] = df['deposit'].map({'yes': 1, 'no': 0})
df['default'] = df['default'].map({'yes': 1, 'no': 0})
df['housing'] = df['housing'].map({'yes': 1, 'no': 0})
df['loan'] = df['loan'].map({'yes': 1, 'no': 0})


std_deposit = round(df['deposit'].std(), 3)
print(f'Стандартное отклонение: {std_deposit}')

default_mean = df['default'].mean()
housing_mean = df['housing'].mean()
loan_mean = df['loan'].mean()

result = round(default_mean + housing_mean + loan_mean, 3)
print(f'Сумма средних: {result}')



### Блок 21: Код выполнения

In [ ]:
nominal_cols = ['job', 'marital', 'contact', 'month', 'poutcome']

df = pd.concat([df, pd.get_dummies(df[nominal_cols], prefix=nominal_cols, drop_first=False)], axis=1)

print(df.columns[-20:])  # Показываем последние 20 столбцов для примера

n_features = df.shape[1] - 1  # Всего столбцов минус столбец deposit
print('Количество признаков:', n_features)
df.drop(columns=nominal_cols, inplace=True)
df = df.drop('age', axis=1)

a_features = df.shape[1] - 1 
print('Количество признаков:', a_features)
print(df)


### Задания 5 и 6

### Блок 22: Код выполнения

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

corr_matrix = df.corr(numeric_only=True)

plt.figure(figsize=(14, 10))
sns.heatmap(corr_matrix, cmap='coolwarm')
plt.title('Корреляционная тепловая карта')
plt.tight_layout()
plt.show()

target_corr = corr_matrix['deposit'].drop('deposit').sort_values(ascending=False)
target_corr.plot(kind='bar', figsize=(8, 6))
plt.title('Корреляция признаков с целевой переменной')
plt.ylabel('Коэффициент корреляции с deposit')
plt.tight_layout()
plt.show()

target_corr = df.corr(numeric_only=True)['deposit'].drop('deposit')

sorted_corr = target_corr.abs().sort_values(ascending=False)

print(sorted_corr)

feature_corr = df.drop(columns=['deposit']).corr(numeric_only=True)



### Задания 7 и 8

### Блок 23: Код выполнения

In [ ]:
X = df.drop(['deposit'], axis=1)
y = df['deposit']
cols = ['balance'] + [col for col in X.columns if col != 'balance']
X = X[cols]
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state = 42, test_size = 0.33)
print(X_test)

### Блок 24: Код выполнения

In [ ]:
print('Размер тестовой выборки:', len(X_test))
mean_target_test = round(y_test.mean(), 2)
print(f'Среднее значение целевой переменной в тестовой выборке: {mean_target_test}')


### Блок 25: Код выполнения

In [ ]:

selector = SelectKBest(score_func=f_classif, k=15)
selector.fit_transform(X_train, y_train)

best_indices = selector.get_support(indices=True)
best_features = X_train.columns[best_indices]

print('Топ-15 лучших признаков:')
for f in best_features:
    print(f)

### Блок 26: Код выполнения

In [ ]:

X_train_selected = X_train[best_features]
X_test_selected = X_test[best_features]

scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train_selected)
X_test_scaled = scaler.transform(X_test_selected)


X_train_scaled = pd.DataFrame(X_train_scaled, columns=best_features, index=X_train.index)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=best_features, index=X_test.index)

print(X_train_scaled.head())
print(X_test_scaled.head())

first_pred_mean = round(X_test_scaled.iloc[:, 0].mean(), 2)
print(f'Среднее значение первого предиктора в тестовой выборке: {first_pred_mean}')

# Часть 4: Решение задачи классификации: логистическая регрессия и решающие деревья

### Блок 27: Код выполнения

In [ ]:
X_train_num = X_train[best_features]
X_test_num = X_test[best_features]

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_num)
X_test_scaled = scaler.transform(X_test_num)

logreg = LogisticRegression(solver='sag', random_state=42, max_iter=1000)
logreg.fit(X_train_scaled, y_train)

y_pred = logreg.predict(X_test_scaled)
accuracy = accuracy_score(y_test, y_pred)

print(f'{accuracy:.2f}') 

### Задания 2,3,4

### Блок 28: Код выполнения

In [ ]:
depths = range(1, 21)

train_scores = []
test_scores = []

for depth in depths:
    tree = DecisionTreeClassifier(criterion='entropy', max_depth=depth, random_state=42)
    tree.fit(X_train[best_features], y_train)
    
    train_acc = accuracy_score(y_train, tree.predict(X_train[best_features]))
    test_acc = accuracy_score(y_test, tree.predict(X_test[best_features]))
    
    train_scores.append(train_acc)
    test_scores.append(test_acc)

plt.figure(figsize=(10, 6))
plt.plot(depths, train_scores, label='Train Accuracy', marker='o')
plt.plot(depths, test_scores, label='Test Accuracy', marker='s')
plt.xlabel('Максимальная глубина дерева')
plt.ylabel('Accuracy')
plt.title('Accuracy vs. Tree Depth')
plt.grid(True)
plt.legend()
plt.xticks(depths)
plt.show()

best_depth = None
best_test_acc = 0
tolerance = 0.01  # допустимая разница между train и test

for i, depth in enumerate(depths):
    train_acc = train_scores[i]
    test_acc = test_scores[i]

    if test_acc >= best_test_acc and (train_acc - test_acc) <= tolerance:
        best_test_acc = test_acc
        best_depth = depth

print(f'✅ Оптимальная глубина дерева: {best_depth}')
print(f'🎯 Accuracy на тесте при этой глубине: {best_test_acc:.2f}')


### Блок 29: Код выполнения

In [ ]:
param_grid = {
    'min_samples_split': [2, 5, 7, 10],
    'max_depth': [3, 5, 7]
}

tree = DecisionTreeClassifier(criterion='entropy', random_state=42)

grid_search = GridSearchCV(tree, param_grid, scoring='f1', cv=5)
grid_search.fit(X_train[best_features], y_train)

best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test[best_features])

f1 = f1_score(y_test, y_pred, average='binary', pos_label=1)


print(f'Лучшие параметры: {grid_search.best_params_}')
print(f'F1 на тесте: {f1:.4f}')
print(f'{f1:.2f}')  # <- этот результат нужно вставить как ответ


# Часть 5: Решение задачи классификации: ансамбли моделей и построение прогноза

### Блок 30: Код выполнения

In [ ]:
rf = RandomForestClassifier(
    n_estimators=100,
    criterion='gini',
    min_samples_leaf=5,
    max_depth=10,
    random_state=42
)
rf.fit(X_train[best_features], y_train)

y_pred = rf.predict(X_test[best_features])

accuracy = accuracy_score(y_test, y_pred)
recall = recall_score(y_test, y_pred, average='binary', pos_label=1)

print(f'Accuracy: {accuracy:.4f}')
print(f'Recall: {recall:.4f}')
print(f'{accuracy:.2f} {recall:.2f}')


### Задания 2 и 3

### Блок 31: Код выполнения

In [ ]:
gb = GradientBoostingClassifier(
    learning_rate=0.05,
    n_estimators=300,
    min_samples_leaf=5,
    max_depth=5,
    random_state=42
)
gb.fit(X_train_scaled, y_train)

y_pred_gb = gb.predict(X_test_scaled)

gb_f1 = round(f1_score(y_test, y_pred_gb), 2)
print('F1:', gb_f1)


### Блок 32: Код выполнения

In [ ]:
estimators = [
    ('tree', DecisionTreeClassifier(criterion='entropy', random_state=42)),
    ('logreg', LogisticRegression(solver='sag', random_state=42, max_iter=1000)),
    ('gb', GradientBoostingClassifier(
        learning_rate=0.05,
        n_estimators=300,
        min_samples_leaf=5,
        max_depth=5,
        random_state=42
    ))
]

final_estimator = LogisticRegression(solver='sag', random_state=42, max_iter=1000)

stacking = StackingClassifier(
    estimators=estimators,
    final_estimator=final_estimator,
    n_jobs=-1
)

stacking.fit(X_train_scaled, y_train)
y_pred_stack = stacking.predict(X_test_scaled)

stack_precision = round(precision_score(y_test, y_pred_stack), 2)
print('Precision:', stack_precision)


### Блок 33: Код выполнения

In [ ]:
model = RandomForestClassifier(
    n_estimators=100,
    criterion='gini',
    min_samples_leaf=5,
    max_depth=10,
    random_state=42
)

model.fit(X_train[best_features], y_train)

importances = model.feature_importances_
features = pd.Series(importances, index=best_features)

target_features = ['contact_unknown', 'duration', 'poutcome_success']
filtered = features[target_features].sort_values(ascending=False)

print("Признаки по убыванию важности:")
print(filtered)

filtered.plot(kind='barh', title='Важность признаков')
plt.gca().invert_yaxis()
plt.show()


### Задания 6,7,8

### Блок 34: Код выполнения

In [ ]:
def objective(trial):
    n_estimators = trial.suggest_int('n_estimators', 100, 200, step=1)
    max_depth = trial.suggest_int('max_depth', 10, 30, step=1)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 2, 10, step=1)

    clf = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        min_samples_leaf=min_samples_leaf,
        random_state=42
    )

    clf.fit(X_train[best_features], y_train)
    y_pred = clf.predict(X_test[best_features])
    return f1_score(y_test, y_pred)

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=30)

best_params = study.best_params
print(f"Лучшие параметры: {best_params}")

best_model = RandomForestClassifier(**best_params, random_state=42)
best_model.fit(X_train[best_features], y_train)
y_pred = best_model.predict(X_test[best_features])

f1 = f1_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)

print(f'F1: {f1:.4f}')
print(f'Recall: {recall:.4f}')
print(f'Accuracy: {accuracy:.4f}')

